## Overview

This notebook provides an exploration of audio data from animals, performing analysis through statistical analysis and visualisation of audio samples

Author: Andrew Kudilczak

## Pre-requisites to run this notebook.

This notebook explores a data set that is stored on the Deakin teams channel here:

[DataSets Folder](https://deakin365.sharepoint.com/:f:/r/sites/DataBytes2/Shared%20Documents/Project%20Echo/DataSets?csf=1&web=1&e=oxQZWj)

This dataset was not uploaded to git to keep the repository light weight.  This notebook assumes the dataset has been mapped to your local drive - see paths below

## Development environment

This notebook requires a conda environment setup in accordance with [Data Science Environment Setup](https://github.com/stephankokkas/Project-Echo/tree/main/src)

In [3]:
# some basic libraries 
from platform import python_version
import pandas as pd
import seaborn as sns
import numpy as np

# plot support
import plotly.express as px
import matplotlib.pyplot as plt

# tensorflow support
import tensorflow as tf
import tensorflow_datasets as tfds

# reading audio datasets
import librosa
import librosa.display
import IPython.display as ipd

# print system information
print('Librosa Version    : ', python_version())
print('TensorFlow Version : ', tf.__version__)
print('Librosa Version    : ', librosa.__version__)

TensorFlow Version 2.10.0
Librosa Version 0.9.2


In [2]:
# set parameters
DATASET_DIR = 'C:/Users/Andrew/OneDrive - Deakin University/DataSets/'